## Setup

In [1]:
# Get raw advent-of-code data
from aocd.models import Puzzle

puzzle = Puzzle(year=2025, day=10)
input_data = puzzle.input_data
example = puzzle.examples[0]

In [2]:
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))

from common.utils.perf_check import check_time

## Part a


### Breadth-first search approach
We can see the lights as bits, and the desired light state as a bitmask, e.g. 0110 -> 6

Pressing a button twice cancels out, so we need to press each button either once or zero times.

Le's start by trying to figure out the smallest combination of lights to press to get to the desired state using a standard breadth-first search (BFS) algorithm.

In [82]:
# Imports
from collections import deque

In [ ]:
# Functions
def parse_input(input_data: str) -> list[tuple[int, list[int], int, list[int]]]:
    """Parse input data into a list of machine settings.

    Each machine has:
        - A number of lights, e.g 4
        - A desired light state, stored as bitmask, e.g. 0110 -> 0 * 8 + 1 * 4 + 1 * 2 + 0 * 1 = 6
        - A list of buttons, stored as bitmasks,  e.g. (0,2) -> 0 * 8 + 1 * 4 + 0 * 2 + 1 * 1 = 5
        - A list of joltages costs for each of the buttons, stored as integers
    """
    lines = input_data.splitlines()
    machines = []
    for line in lines:
        # Break the string in 3 parts
        lights_str, rest = line.split("] (")
        buttons_str, joltages_str = rest.split(") {")

        # Parse lights into integers and then into bitmask
        lights_int = [c == "#" for c in lights_str[1:]]
        n_lights = len(lights_int)
        lights_target = sum((v & 1) << i for i, v in enumerate(lights_int))

        # Parse buttons into integers and then into bitmask
        buttons_int = [[*map(int, button_str.split(","))] for button_str in buttons_str.split(") (")]
        buttons_bitmask = []
        for button in buttons_int:
            bitmask = 0
            for toggle in button:
                # Add the binary form of the toggle to the button's bitmask
                bitmask |= 1 << toggle
            buttons_bitmask.append(bitmask)

        # Parse joltages
        joltages = [int(j) for j in joltages_str[:-1].split(",")]

        # Save values to machine tuple
        machines.append((lights_target, buttons_bitmask, n_lights, joltages))

    return machines


def min_presses_bfs(lights_target: int, buttons: list[int]) -> int | None:
    """Find the minimum number of presses for a desired light state and list of buttons (as bitmasks) using BFS."""
    # Handle null-state
    if lights_target == 0:
        return 0

    # Initialize BFS queue and track visited states
    q = deque([(0, 0)])
    seen = {0}

    while q:
        # Unpack the next state in the queue
        lights_state, n_presses = q.popleft()

        for button in buttons:
            # 'Press' the button (equivalent to XOR operation)
            next_lights_state = lights_state ^ button

            if next_lights_state == lights_target:
                # Found the target lights state, include the next press in the answer
                return n_presses + 1

            if next_lights_state not in seen:
                # Add the next state to the visited states and to the queue
                seen.add(next_lights_state)
                q.append((next_lights_state, n_presses + 1))

    # Return None if no combination of button presses results in the target lights state
    return None


def solve_a_bfs(input_data: str) -> int:
    """Find the sum of the minimum number of presses for each machine using a BFS algorithm."""
    total = 0
    for lights_target, buttons, _, _ in parse_input(input_data):
        presses = min_presses_bfs(lights_target, buttons)
        if presses is None:
            msg = f"No solution found for light target {lights_target} with buttons {buttons}."
            raise ValueError(msg)
        total += presses
    return total


In [ ]:
# Correctness check
str(solve_a_bfs(example.input_data)) == example.answer_a

True

In [119]:
# Performance check
bfs_time_a = check_time(solve_a_bfs, input_data)
print(f"The BFS approach takes {bfs_time_a:.1f} ms per run.")


The BFS approach takes 3.8 ms per run.


### Linear algebra
BFS works fine, but I feel like we can be more analytical. Linear algebra it is! 

If we see the light state as a target vector `l` and the buttons as a matrix `B`, we can see the amount of presses per button as `x`, and we need to solve:
```
A * x = b
```

However, we need to ensure the whole system is modulo 2, i.e. all numbers can only be 0 or 1. For this, we can use Galois Fields; specifically the Galois Field of two elements (GF(2)). We need to perform Gaussian Elimination (GE) to find the smallest solution in this GF(2) space.

Disclaimer: this is quite complex and I mostly just adapted existing implementations of the GF(2) GE algorithm , described here: https://www.cs.umd.edu/~gasarch/TOPICS/factoring/fastgauss.pdf

In [145]:
# Functions
def min_presses_gf2(n_lights: int, lights_target: int, buttons: list[int]) -> None | int:
    """Find the minimum number of presses for a desired light state using Gaussian Elimination in the GF(2) field."""
    # Set 'm' to the number of buttons
    m = len(buttons)

    # Augmented matrix: rows are lights, columns are buttons + 1 target col
    # Each row is an integer bitmask. Bits 0..m-1 are button toggles, Bit m is target.
    rows = []
    for i in range(n_lights):
        row = 0
        for j, b_mask in enumerate(buttons):
            if (b_mask >> i) & 1:
                row |= 1 << j  # Button j affects Light i
        if (lights_target >> i) & 1:
            row |= 1 << m  # Light i needs to be ON
        rows.append(row)

    # Gaussian Elimination
    pivot_row = 0
    pivot_cols = []
    for j in range(m):
        if pivot_row >= n_lights:
            break

        # Find a row below the current pivot_row that has a 1 in column j
        sel = next((i for i in range(pivot_row, n_lights) if (rows[i] >> j) & 1), None)
        if sel is None:
            continue  # No pivot in this column, it's a "free variable"

        # Swap the found row into the current pivot position
        rows[pivot_row], rows[sel] = rows[sel], rows[pivot_row]

        # XOR this row into all other rows to clear column j
        for i in range(n_lights):
            if i != pivot_row and (rows[i] >> j) & 1:
                rows[i] ^= rows[pivot_row]

        pivot_cols.append(j)
        pivot_row += 1

    # Solvability check: i a row is 0=1 (bits 0..m-1 are 0, but bit m is 1), it's impossible.
    for i in range(pivot_row, n_lights):
        if (rows[i] >> m) & 1:
            return None

    # Optimize over the free variables, columns that didn't get a pivot are "flexible".
    free_cols = [j for j in range(m) if j not in pivot_cols]
    best_presses = float("inf")

    # Brute force 2^N combinations of free variables (this is fine for N<=10, which is the case in our input data)
    for i in range(1 << len(free_cols)):
        current_press_mask = 0
        for idx, col in enumerate(free_cols):
            if (i >> idx) & 1:
                current_press_mask |= 1 << col

        # For this specific setup of free variables, solve for the forced pivot variables
        for r_idx, p_col in enumerate(pivot_cols):
            # The equation is: PivotVariable ^ (Sum of FreeVariables in Row) = TargetBit
            target_bit = (rows[r_idx] >> m) & 1
            # (rows[r_idx] & current_press_mask).bit_count() % 2 is the XOR sum of free vars
            free_vars_xor_sum = (rows[r_idx] & current_press_mask).bit_count() % 2

            if target_bit ^ free_vars_xor_sum:
                current_press_mask |= 1 << p_col

        best_presses = min(best_presses, (current_press_mask).bit_count())

    return int(best_presses) if best_presses != float("inf") else None


def solve_a_gf2(input_data: str) -> int:
    """Find the sum of the minimum number of presses for each machine using Gaussian Elimination in the GF(2) field."""
    total = 0
    for lights_target, buttons, n_lights, _ in parse_input(input_data):
        presses = min_presses_gf2(n_lights, lights_target, buttons)
        if presses is None:
            msg = f"No solution found for light target {lights_target} with buttons {buttons}."
            raise ValueError(msg)
        total += presses
    return total


In [132]:
# Correctness check
str(solve_a_gf2(example.input_data)) == example.answer_a

True

In [146]:
# Performance check
gf2_time_a = check_time(solve_a_gf2, input_data)
print(f"The GF2 GE approach takes {gf2_time_a:.1f} ms per run.")
print(f"This is {bfs_time_a / gf2_time_a:.1f} x faster than the BFS approach.")


The GF2 GE approach takes 2.7 ms per run.
This is 1.4 x faster than the BFS approach.


In [123]:
# Submit answer
puzzle.answer_a = solve_a_gf2(input_data)

That's the right answer!  You are one gold star closer to decorating the North Pole. [Continue to Part Two]


## Part b
